# Analysis Q-Peak Algorithms

## Setup and Helper Functions

### Imports

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.data_handling import compute_pep_performance_metrics, get_error_by_group
from pepbench.datasets import EmpkinsDataset, GuardianDataset
from pepbench.export import convert_to_latex, create_algorithm_result_table
from pepbench.io import load_challenge_results_from_folder
from pepbench.plotting.algorithms import (
    plot_q_peak_extraction_martinez2004_neurokit,
)
from pepbench.plotting.results import (
    boxplot_algorithm_performance,
    plot_q_wave_detection_waveform_detailed_comparison,
    regplot_error_heart_rate,
    residual_plot_pep,
    residual_plot_pep_heart_rate,
    residual_plot_pep_participant,
    residual_plot_pep_phase,
    violinplot_algorithm_performance,
)
from pepbench.utils import rename_algorithms, rename_metrics

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path("../../")

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

### Input Paths

In [ ]:
result_path = root_path.joinpath("results")

### Output Paths

In [ ]:
paper_path = json.load(root_path.joinpath("paper_path.json").open(encoding="utf-8"))["paper_path"]
paper_path = Path(paper_path)

export_path = root_path.joinpath("exports")
img_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("stats")

img_path_paper = paper_path.joinpath("img")
tab_path_paper = paper_path.joinpath("tab")
suppl_img_path_paper = paper_path.joinpath("supplementary_material/img")
suppl_tab_path_paper = paper_path.joinpath("supplementary_material/tab")

bp.utils.file_handling.mkdirs(
    [
        result_path,
        export_path,
        img_path,
        stats_path,
        img_path_paper,
        tab_path_paper,
        suppl_img_path_paper,
        suppl_tab_path_paper,
    ]
)

In [ ]:
algo_levels = ["q_peak_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(
    zip(algo_levels, ["Q-Peak Algorithm", "B-Point Algorithm", "Outlier Correction"], strict=False)
)

## EmpkinS Dataset

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_q_peak"),
    index_cols_per_sample=["participant", "condition", "phase"],
)

In [ ]:
results_per_sample_empkins = results_empkins.per_sample.droplevel([1, 2])
results_agg_total_empkins = results_empkins.agg_total.droplevel([1, 2])
results_per_sample_empkins.head()

In [ ]:
metrics_empkins = compute_pep_performance_metrics(results_per_sample_empkins, num_heartbeats=results_agg_total_empkins)
metrics_empkins.style.highlight_min(subset=["Mean Absolute Error [ms]"], props="background-color: LightGreen;")

#### Latex Export

In [ ]:
result_table = create_algorithm_result_table(metrics_empkins)
result_table.index = result_table.index.str.replace(" (", r"\,(", regex=False)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=True,
    column_header_bold=True,
    column_format="p{1.60cm}S[table-format=1.1(2)]S[table-format=1.1(2)]S[table-format=1.1(2)]p{1.30cm}",
    caption=r"Results of the Q-peak extraction algorithms on the \textit{EmpkinS Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order.",
    label="tab:q_peak_results_empkins",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Invalid", r"{\bfseries Invalid")
latex_output = latex_output.replace(r"{\bfseries Algorithm}", r"{\bfseries Q-peak\newline Algortihm}")

tab_path_paper.joinpath("tab_q_peak_results_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

### Plots

In [ ]:
selected_algos_for_plotting_empkins = ["martinez2004", "vanlien2013-34-ms", "vanlien2013-40-ms", "forouzanfar2018"]

In [ ]:
results_empkins_plot = results_per_sample_empkins.reindex(selected_algos_for_plotting_empkins, level="q_peak_algorithm")
results_empkins_plot.head()

#### Absolute Error

In [ ]:
fig, ax = boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

fig.savefig(img_path.joinpath("img_boxplot_q_peak_algorithms_mae_empkins.pdf"), transparent=True)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

#### Absolute Error (with and without Outlier)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 3))

boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=True,
    width=0.9,
    title="With Outlier",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="Without Outlier",
    fig=fig,
    ax=axs[1],
)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_q_peak_algorithms_mae_with_without_outlier_empkins.pdf"), transparent=True)

#### Error

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins_plot,
    metric="error_per_sample_ms",
    figsize=(6, 5),
)

#### Error per Participant

In [ ]:
error_per_participant_empkins = get_error_by_group(results_per_sample_empkins, grouper="participant")
error_per_participant_empkins = error_per_participant_empkins.reindex(
    selected_algos_for_plotting_empkins, level="q_peak_algorithm", axis=1
)
error_per_participant_empkins = error_per_participant_empkins.round(2)
error_per_participant_empkins = error_per_participant_empkins.rename(columns=rename_algorithms).rename(
    columns=rename_metrics
)
error_per_participant_empkins.style.highlight_max(props="background-color: Pink;")

In [ ]:
latex_output = convert_to_latex(
    error_per_participant_empkins.style.highlight_max(props="background-color: Pink;").format_index(
        escape="latex", axis=0
    ),
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{3.0cm}" + "S[table-format=2.2]" * len(error_per_participant_empkins.columns),
    caption=r"Mean Abolute Error of selected Q-peak extraction algorithms on the \textit{EmpkinS Dataset} per participant. The values with the highest errors are highlighted in red.",
    label="tab:q_peak_results_per_participant_empkins",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"\begin{table}[ht]", r"\begin{table}[ht]\small")
latex_output = latex_output.replace(r"q_peak_algorithm", r"\bfseries Q-peak Algorithm")
latex_output = latex_output.replace(r"{participant}", r"{Participant}")
latex_output = latex_output.replace(r"{metric}", r"{}")
latex_output = latex_output.replace(r"{\bfseries mean}", r"{Mean}")
latex_output = latex_output.replace(r"{\bfseries std}", r"{SD}")
latex_output = latex_output.replace(r"{std}", r"{SD}")
latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")
# latex_output = latex_output.replace(r"{\bfseries Algorithm}", r"{\bfseries Q-peak\newline Algortihm}")

suppl_tab_path_paper.joinpath("tab_q_peak_results_per_participant_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Residual Plots

In [ ]:
selected_algos_for_residual_empkins = ["martinez2004", "vanlien2013-34-ms", "forouzanfar2018"]

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep(results_per_sample_empkins, algo, annotate_fontsize="small", annotate_bbox=True, ax=axs[i])
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-75, 50)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_participant(
        results_per_sample_empkins, algo, alpha=0.5, ax=axs[i], annotate_fontsize="small", annotate_bbox=True
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-75, 50)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_per_participant_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_phase(
        results_per_sample_empkins,
        algo,
        alpha=0.5,
        ax=axs[i],
        annotate_fontsize="small",
        annotate_bbox=True,
        show_legend=True,
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-75, 50)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_per_phase_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_heart_rate(
        results_per_sample_empkins, algo, alpha=0.5, annotate_fontsize="small", annotate_bbox=True, ax=axs[i]
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-75, 50)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_heart_rate_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 4), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    regplot_error_heart_rate(
        results_per_sample_empkins,
        algo,
        error_metric="absolute_error_per_sample_ms",
        add_corr_coeff=True,
        ax=axs[i],
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(axs[0].get_ylim()[0], 80)

fig.tight_layout()
for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_q_peak_error_heart_rate_empkins.pdf"), transparent=True)

## Deeper Investigation

In [ ]:
subset_01 = dataset_empkins.get_subset(participant="VP_001", condition="ftsst", phase="Pause_1")
subset_02 = dataset_empkins.get_subset(participant="VP_002", condition="ftsst", phase="Pause_1")

In [ ]:
fig, axs = plot_q_wave_detection_waveform_detailed_comparison(
    subset_01,
    subset_02,
    base_plot_func=plot_q_peak_extraction_martinez2004_neurokit,
    plot_func_01_params={"heartbeat_subset": (1, 9)},
    plot_func_02_params={"heartbeat_subset": (1, 9)},
    ax_inset_01_params={"xmin": 1.35, "xmax": 1.6, "bounds": [1.01, -0.1, 0.33, 1.2]},
    ax_inset_02_params={"xmin": 1.35, "xmax": 1.6, "bounds": [1.01, -0.1, 0.33, 1.2]},
    datapoint_01_name="VP_001",
    datapoint_02_name="VP_002",
)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_q_peak_detection_waveform_examples_empkins.pdf"), transparent=True)

## Guardian Dataset

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

In [ ]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_q_peak"),
    index_cols_per_sample=["participant", "phase"],
)

In [ ]:
results_per_sample_guardian = results_guardian.per_sample.droplevel([1, 2])
results_agg_total_guardian = results_guardian.agg_total.droplevel([1, 2])
results_per_sample_guardian.head()

In [ ]:
metrics_guardian = compute_pep_performance_metrics(
    results_per_sample_guardian, num_heartbeats=results_agg_total_guardian
)
metrics_guardian.style.highlight_min(subset=["Mean Absolute Error [ms]"], props="background-color: LightGreen;")

#### Latex Export

In [ ]:
result_table = create_algorithm_result_table(metrics_guardian)
result_table.index = result_table.index.str.replace(" (", r"\,(", regex=False)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=True,
    column_header_bold=True,
    column_format="p{1.60cm}S[table-format=1.1(2)]S[table-format=1.1(2)]S[table-format=1.1(2)]p{1.30cm}",
    caption=r"Results of the Q-peak extraction algorithms on the \textit{Guardian Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order.",
    label="tab:q_peak_results_guardian",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Invalid", r"{\bfseries Invalid")
latex_output = latex_output.replace(r"{\bfseries Algorithm}", r"{\bfseries Q-peak\newline Algortihm}")

tab_path_paper.joinpath("tab_q_peak_results_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

## Plots

In [ ]:
selected_algos_for_plotting_guardian = ["martinez2004", "vanlien2013-32-ms", "vanlien2013-40-ms", "forouzanfar2018"]

In [ ]:
results_guardian_plot = results_per_sample_guardian.reindex(
    selected_algos_for_plotting_guardian, level="q_peak_algorithm"
)
results_guardian_plot.head()

#### Absolute Error

In [ ]:
fig, ax = boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

fig.savefig(img_path.joinpath("img_boxplot_q_peak_algorithms_mae_guardian.pdf"), transparent=True)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

#### Absolute Error (with and without Outlier)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 3))

boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=True,
    width=0.9,
    title="With Outlier",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="Without Outlier",
    fig=fig,
    ax=axs[1],
)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_q_peak_algorithms_mae_with_without_outlier_guardian.pdf"), transparent=True)

#### Error

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_guardian_plot,
    metric="error_per_sample_ms",
    figsize=(6, 5),
)

#### Error per Participant

In [ ]:
error_per_participant_guardian = get_error_by_group(results_per_sample_guardian, grouper="participant")
error_per_participant_guardian = error_per_participant_guardian.reindex(
    selected_algos_for_plotting_guardian, level="q_peak_algorithm", axis=1
)
error_per_participant_guardian = error_per_participant_guardian.round(2)
error_per_participant_guardian = error_per_participant_guardian.rename(columns=rename_algorithms).rename(
    columns=rename_metrics
)
error_per_participant_guardian.style.highlight_max(props="background-color: Pink;")

In [ ]:
latex_output = convert_to_latex(
    error_per_participant_guardian.style.highlight_max(props="background-color: Pink;").format_index(
        escape="latex", axis=0
    ),
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{3.0cm}" + "S[table-format=2.2]" * len(error_per_participant_guardian.columns),
    caption=r"Mean Abolute Error of selected Q-peak extraction algorithms on the \textit{Guardian Dataset} per participant. The values with the highest errors are highlighted in red.",
    label="tab:q_peak_results_per_participant_guardian",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"\begin{table}[ht]", r"\begin{table}[ht]\small")
latex_output = latex_output.replace(r"q_peak_algorithm", r"\bfseries Q-peak Algorithm")
latex_output = latex_output.replace(r"{participant}", r"{Participant}")
latex_output = latex_output.replace(r"{metric}", r"{}")
latex_output = latex_output.replace(r"{\bfseries mean}", r"{Mean}")
latex_output = latex_output.replace(r"{\bfseries std}", r"{SD}")
latex_output = latex_output.replace(r"{std}", r"{SD}")
latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")

suppl_tab_path_paper.joinpath("tab_q_peak_results_per_participant_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Residual Plots

In [ ]:
selected_algos_for_residual_guardian = ["martinez2004", "vanlien2013-32-ms", "forouzanfar2018"]

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep(results_per_sample_guardian, algo, annotate_fontsize="small", annotate_bbox=True, ax=axs[i])
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-115, 40)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_participant(
        results_per_sample_guardian, algo, alpha=0.5, annotate_fontsize="small", annotate_bbox=True, ax=axs[i]
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-115, 40)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_per_participant_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_phase(
        results_per_sample_guardian,
        algo,
        alpha=0.5,
        ax=axs[i],
        annotate_fontsize="small",
        annotate_bbox=True,
        rect=(0, 0, 0.85, 1),
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-115, 40)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_per_phase_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_empkins), figsize=(12, 5), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_empkins):
    residual_plot_pep_heart_rate(
        results_per_sample_guardian,
        algo,
        alpha=0.5,
        ax=axs[i],
        annotate_fontsize="small",
        annotate_bbox=True,
        rect=(0, 0, 0.85, 1),
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(-115, 40)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_q_peak_algorithms_heart_rate_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=len(selected_algos_for_residual_guardian), figsize=(12, 4), sharey=True)

for i, algo in enumerate(selected_algos_for_residual_guardian):
    regplot_error_heart_rate(
        results_per_sample_guardian,
        algo,
        error_metric="absolute_error_per_sample_ms",
        add_corr_coeff=True,
        ax=axs[i],
    )
    if i != 0:
        axs[i].set_ylabel(None)

axs[0].set_ylim(axs[0].get_ylim()[0], 150)

fig.tight_layout()
for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_q_peak_error_heart_rate_guardian.pdf"), transparent=True)

## Deeper Investigation

In [ ]:
subset_01 = dataset_guardian.get_subset(participant="GDN0007", phase="Pause")
subset_02 = dataset_guardian.get_subset(participant="GDN0009", phase="Pause")

In [ ]:
fig, axs = plot_q_wave_detection_waveform_detailed_comparison(
    subset_01,
    subset_02,
    base_plot_func=plot_q_peak_extraction_martinez2004_neurokit,
    plot_func_01_params={"heartbeat_subset": (1, 9)},
    plot_func_02_params={"heartbeat_subset": (1, 9)},
    ax_inset_01_params={"xmin": 2.5, "xmax": 2.75, "bounds": [1.01, -0.1, 0.33, 1.2]},
    ax_inset_02_params={"xmin": 2.25, "xmax": 2.5, "bounds": [1.01, -0.1, 0.33, 1.2]},
    datapoint_01_name="GDN0007",
    datapoint_02_name="GDN0009",
)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_q_peak_detection_waveform_examples_guardian.pdf"), transparent=True)

## Combined Figures

### Absolute Error

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 2), sharey=True)

boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="EmpkinS Dataset – Q-Peak Detection",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="Guardian Dataset – Q-Peak Detection",
    fig=fig,
    ax=axs[1],
)
for ax in axs:
    ax.set_xlabel(None)

fig.tight_layout()

for path in [img_path, img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_q_peak_algorithms_mae_combined.pdf"), transparent=True)